<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/Version_1_datagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#connect drive with colab notebook/

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


https://github.com/keras-team/keras/issues/8130
https://stackoverflow.com/questions/56796331/how-to-input-multiple-images-with-flow-from-dataframe-in-keras

In [2]:
#following are libraries to unpack 7z file
!pip install patool 
!pip install pyunpack
from pyunpack import Archive

     |████████████████████████████████| 81kB 3.2MB/s 
  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5162 sha256=a7a2a86dcc10d37dc12a2850bf7e3e18fd82ad7a45d416b84493834d4dfb62ee
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack


In [0]:
import pandas as pd
import numpy as np
import cv2

In [0]:

#create a folder name as data
!mkdir data

In [0]:
#path where our data is saved
path='/content/drive/My Drive/ghina/thesis/'

In [0]:
#extract the 7z file in data folder we created above
#this not only extract the data but move it from drive to colab for efficinet processing
#it will take some time
Archive(path+'ODIR-5K_Training_Images.7z').extractall('data')

In [0]:
#read the annottion file
annotation=pd.read_excel(path+'ODIR-5K_Training_Annotations(Updated)_V2.xlsx')

In [0]:

#normal (N), diabetes (D), glaucoma (G), cataract (C), AMD (A), hypertension (H), myopia (M) and other diseases/abnormalities (O)

In [9]:
#print first 5 rows of annotation file
annotation=annotation.sample(frac=1)
annotation.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
1226,1311,67,Female,1311_left.jpg,1311_right.jpg,normal fundus,vitreous degeneration,0,0,0,0,0,0,0,1
896,898,77,Female,898_left.jpg,898_right.jpg,moderate non proliferative retinopathy，epireti...,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
738,740,56,Male,740_left.jpg,740_right.jpg,normal fundus,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0
2362,2985,61,Female,2985_left.jpg,2985_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
1085,1125,37,Female,1125_left.jpg,1125_right.jpg,dry age-related macular degeneration,dry age-related macular degeneration,0,0,0,0,1,0,0,0


In [0]:
train_annotation=annotation[0:2500]
val_annotation=annotation[2500:3500]


In [0]:
from keras_preprocessing.image import ImageDataGenerator

train_data_gen= ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest'
)

val_data_gen= ImageDataGenerator(
    rescale=1/255
    
)

In [0]:

def get_flow_from_dataframe(generator, dataframe,directory_path,
                            image_shape=(224, 224),
                            #subset='training',
                             batch_size=50):
    train_generator_1 = generator.flow_from_dataframe(dataframe,
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Left-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )

    train_generator_2 = generator.flow_from_dataframe(dataframe, 
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )
    while True:
        x_1 = train_generator_1.next()
        x_2 = train_generator_2.next()

        yield [x_1[0], x_2[0]], x_1[1]

In [37]:
train_generator_2 = val_data_gen.flow_from_dataframe(val_annotation, 
                                                      directory="data/ODIR-5K_Training_Dataset/",
                                                      target_size=(224,224),
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      )

Found 1000 validated image filenames.


In [0]:
batch_size=50
train_gen = get_flow_from_dataframe(train_data_gen, train_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
valid_gen = get_flow_from_dataframe(val_data_gen, val_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)

In [0]:
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
inp1= Input(shape=(224, 224, 3))
inp2= Input(shape=(224, 224, 3))

X1=BatchNormalization()(inp1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=MaxPooling2D(pool_size=(2, 2))(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=GlobalMaxPooling2D()(X1)


X2=BatchNormalization()(inp2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=MaxPooling2D(pool_size=(2, 2))(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=GlobalMaxPooling2D()(X2)

X=concatenate([X1,X2])
X=Dropout(0.25)(X)
X=Dense(512,activation='relu')(X)
X=Dropout(0.25)(X)
out=Dense(8, activation='sigmoid')(X)

model=Model(inputs=[inp1,inp2],outputs=out)

model.compile('adam',loss="binary_crossentropy",metrics=["accuracy"])

In [0]:
step_size_train=2500/50
step_size_val=1000/50


In [0]:
model.fit_generator(train_gen, epochs=5,
                        steps_per_epoch=step_size_train,
                        use_multiprocessing=True,
                        validation_data=valid_gen,
                        validation_steps=step_size_val,
                        verbose=2
                    
                        )

Epoch 1/5
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 2500 validated image filenames.
Found 2500 validated image filenames.


In [0]:
y_pred = model.predict_generator(val_generator,steps=step_size_valid)


In [0]:
y_pred.shape

(992, 8)

In [0]:
y_pred[0]

array([0.35950512, 0.29279578, 0.10425806, 0.10760719, 0.06777266,
       0.05234444, 0.08795124, 0.2723663 ], dtype=float32)